<a href="https://colab.research.google.com/github/god7i11a/pynb/blob/main/SST_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources
# https://climatereanalyzer.org/clim/sst_daily/
# https://www.pyngl.ucar.edu/
# https://www.ncl.ucar.edu/External/

# why $5\sigma$ ???:    # https://home.cern/resources/faqs/five-sigma

# TODOs:
## explore other ways of quantifying variations besides daily distribution around the 20 year mean.

In [ ]:
import os
import re
import sys

try:
    from google.colab import userdata, output
    output.enable_custom_widget_manager()
    ! pip install ipympl
    ! sudo apt-get install cm-super dvipng texlive-latex-extra texlive-latex-recommended
    COLAB = True
except ModuleNotFoundError:
    COLAB=False

get_ipython().run_line_magic('matplotlib', 'ipympl')

from locale import atof
import math
import datetime
import ipywidgets as widgets
from numpy import datetime_as_string, inf, array, linspace, sin, vstack, arange
import branca.colormap as cm
from pandas import DataFrame, to_datetime, Series, concat, isna, read_json, timedelta_range
from matplotlib import pyplot as plt, colors


plt.rcParams['text.usetex'] = True

# get the latest data

# global average   https://climatereanalyzer.org/clim/t2_daily/json/era5_world_t2_day.json
# gridded data https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/

In [ ]:
fileN = 'oisst2.1_world2_sst_day.json'
URL0 = 'https://climatereanalyzer.org/clim/sst_daily/json/'
end_offset=-4
URL1 = 'https://climatereanalyzer.org/clim/sst_daily/json_2clim/'
end_offset=-3
df = read_json(path_or_buf=f"{URL1}/{fileN}")
df.set_index("name", inplace=True)
df = DataFrame(df["data"].to_list(), columns=list(range(1, 367)), index=df.index)
last_year = df.index.to_list()[end_offset]
day_of_year = df.loc[last_year, :].dropna().index.to_list()[-1]
last_date=(datetime.datetime(int(last_year), 1, 1) + datetime.timedelta(day_of_year - 1)).strftime('%m/%d/%Y')
print(f'latest data point is {last_date}')

# compute mean and standard deviations and display raw data

In [ ]:
cdf=df.T.loc[:, '1981':'2024']
cdf['mean'] = cdf.loc[:,'1982':'2011'].mean(axis=1, skipna=True)
cdf['std'] = cdf.loc[:,'1982':'2011'].std(axis=1, skipna=True)
cdf['finalmean'] = cdf.loc[:,'2012':'2024'].mean(axis=1, skipna=True)
cdf['finalstd'] = cdf.loc[:,'2012':'2024'].std(axis=1, skipna=True)
cdf['mean+5std'] = cdf['mean']+5*cdf['std']
cdf['mean+5finalstd'] = cdf['mean']+5*cdf['finalstd']
cdf['mean+2std'] = cdf['mean']+2*cdf['std']
cdf['mean-2std'] = cdf['mean']-2*cdf['std']
cdf

# Colormaps
# https://colorcet.holoviz.org/
# https://cmasher.readthedocs.io/user/usage.html#accessing-colormaps

In [ ]:
gradient = linspace(0, 1, 256)
gradient = vstack((gradient, gradient))
cmaps = {'Cyclic': ['twilight', 'twilight_shifted', 'hsv'],
         'Perceptually Uniform Seq': ['viridis', 'plasma', 'inferno', 'magma', 'cividis'],
         'Seq': ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                        'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                        'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'],
         'Seq2': ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
                          'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
                          'hot', 'afmhot', 'gist_heat', 'copper'],
         'Divirgent': ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
                       'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'],
         'Qualitative': ['Pastel1', 'Pastel2', 'Paired', 'Accent',
                         'Dark2', 'Set1', 'Set2', 'Set3',
                         'tab10', 'tab20', 'tab20b', 'tab20c'],
         'Misc': ['flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
                           'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
                           'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
                           'gist_ncar']}



cmap_types = list(cmaps.keys())
cmap_names= [cname  for ctype in cmap_types for cname in cmaps[ctype] ]

w = widgets.Dropdown(
    options= cmap_names,
    value='jet',
    description='ColorMap Choice:',
)

display(w)

## graph

In [ ]:
SHOWSIG = True
ACCENTS=True

cmapStr = w.value
if SHOWSIG:
    fig = plt.figure( figsize=(16,14), clear=True, num=4, layout='constrained')
    (ax,ax1) = fig.subplots(2,1, sharex=True, height_ratios=[.75,.25])
else:
  fig, ax= plt.subplots(1,1, figsize=(16,16), num=4, clear=True)

ax.plot(cdf.index,cdf['mean'], 'b', lw=4, label='1982-2011 mean')
ax.plot(cdf.index, cdf['mean+2std'], 'k', lw=4, label=r'1982-2011 mean $\pm 2\sigma$')
ax.plot(cdf.index, cdf['mean-2std'], 'k', lw=4)
ax.plot(cdf.index, cdf['mean+5std'], 'r', lw=4, label=r'1982-2011 mean +$5\sigma$')

yearL = range(1981, int(last_year)+1)
num_colors = len(yearL)
cmap = plt.get_cmap(cmapStr,num_colors)

_skip = 10
ms=5
hl_years = {2009: 'o', 2010:'s', 2011:'^', 2012: 'v', 2013:'*', 2014: 'D', 2023:'x', 2024:'D'}

for i,_year in enumerate(yearL):
  ax.plot(cdf.index, cdf.loc[:,f'{_year}'], c=cmap(i), lw=1)
  if ACCENTS:
    if _year in hl_years.keys():
      day_num = cdf.index.to_list()[0::_skip]
      data = cdf[f'{_year}'].iloc[0::_skip]
      ax.plot(day_num, data, hl_years[_year], c=cmap(i), ms=ms, label=f'{_year}')

norm = colors.Normalize(vmin=1981,vmax=yearL[-1])
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# check that the color chosen in is the middle of the color segment
plt.colorbar(sm, ax=ax, ticks=linspace(1981,yearL[-1], num_colors), boundaries=arange(1981-.5,yearL[-1]+1,1))

ax.set_ylabel(r'$T\ \  (^{\circ} C)$', fontsize='xx-large')
ax.set_title(f'1981-{last_year} SST daily average', fontsize='xx-large')
ax.text(275, 19.6,  f'\\it{{data current to {last_date}}}', fontsize='xx-large')
ax.set_xlim(0,365)
ax.legend(ncols=4)

if SHOWSIG:
  ax1.plot(cdf.index,cdf['std'], 'b.', ms=4)
  ax1.set_ylabel(r'$\sigma\ \  (^{\circ} C)$', fontsize='xx-large')
  ax1.set_xlabel(r'day \#', fontsize='xx-large')
  ax1.set_title(r'daily $\sigma$', fontsize='xx-large')
  #plt.tight_layout()
else:
  ax.set_xlabel(r'day \#', fontsize='xx-large')

plt.savefig('SST-color.png')
plt.show()

# SST jumps

In [ ]:
SHOWSIG = True
ACCENTS=True

cmapStr = w.value
if SHOWSIG:
    fig = plt.figure( figsize=(16,14), clear=True, num=6, layout='constrained')
    (ax,ax1) = fig.subplots(2,1, sharex=True, height_ratios=[.75,.25])
else:
  fig, ax= plt.subplots(1,1, figsize=(16,16), num=4, clear=True)

ax.plot(cdf.index,cdf['mean'], 'b', lw=4, label='1982-2011 mean')
ax.plot(cdf.index, cdf['mean+2std'], 'k', lw=4, label=r'1982-2011 mean $\pm 2\sigma$')
ax.plot(cdf.index, cdf['mean-2std'], 'k', lw=4)
ax.plot(cdf.index, cdf['mean+5std'], 'r', lw=4, label=r'1982-2011 mean +$5\sigma$')
ax.plot(cdf.index, cdf['mean+5finalstd'], 'g', lw=4, label=r'1982-2011 mean +$5\sigma_{final}$')

yearL = range(1981, int(last_year)+1)
num_colors = len(yearL)
cmap = plt.get_cmap(cmapStr,num_colors)

_skip = 10
ms=5
year_choice = (1996, 1997, 1998, 1999, 2022, 2023, 2024)
hl_years = {1996: 'D', 1997: 'x', 1998:'s', 1999: '^', 2022: 'D', 2023:'x', 2024: 's'} #  2013:'*', 2014: 'D', 2023:'x', 2024:'D'

for i,_year in enumerate(yearL):
    if _year in year_choice:
      ax.plot(cdf.index, cdf.loc[:,f'{_year}'], c=cmap(i), lw=1)
    if _year in hl_years.keys():
      day_num = cdf.index.to_list()[0::_skip]
      data = cdf[f'{_year}'].iloc[0::_skip]
      ax.plot(day_num, data, hl_years[_year], c=cmap(i), ms=ms, label=f'{_year}')


ax.set_ylabel(r'$T\ \  (^{\circ} C)$', fontsize='xx-large')
ax.set_title('SST jumps of 1997 and 2023', fontsize='xx-large')
ax.text(275, 19.6,  f'\\it{{data current to {last_date}}}', fontsize='xx-large')
ax.set_xlim(0,365)
ax.legend(ncols=3)

if SHOWSIG:
  ax1.plot(cdf.index,cdf['std'], 'b.', ms=4)
  ax1.plot(cdf.index, cdf['finalstd'], 'r', ms=4)
  ax1.set_ylabel(r'$\sigma\ \  (^{\circ} C)$', fontsize='xx-large')
  ax1.set_xlabel(r'day \#', fontsize='xx-large')
  ax1.set_title(r'daily $\sigma$', fontsize='xx-large')
  #plt.tight_layout()
else:
  ax.set_xlabel(r'day \#', fontsize='xx-large')

plt.savefig('SST-color-jumps.png')
plt.show()

In [ ]:
fig_all_T, ax_all_T = plt.subplots(1,1, figsize=(10,8), clear=True, num=5)
for i,_year in enumerate(yearL):
  ax_all_T.plot(array(cdf.index.to_list())/365.+i+1981, cdf[f'{_year}'], c=cmap(i), lw=1)
plt.colorbar(sm, ax=ax_all_T, ticks=linspace(1981,yearL[-1], num_colors), boundaries=arange(1981-.5,yearL[-1]+1,1))
plt.savefig('one-curve.png')
plt.show()

# Just checking my stuff against Climate Reanalyzer graph

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(16,8), num=2, clear=True)
ax.plot(df.T.index, df.T['1982-2011 mean'])
ax.plot(df.T.index, df.T['plus 2σ'])
ax.plot(df.T.index, df.T['minus 2σ'])
ax.plot(cdf.index, cdf['mean+5std'], 'r-.', lw=4)
ax.plot(cdf.index, cdf['mean+2std'], 'k-.', lw=4)
ax.plot(cdf.index, cdf['mean-2std'], 'k-.', lw=4)
ax.plot(cdf.index,cdf['mean'], 'b-.', lw=4)

ax.plot(cdf.index, cdf['mean+5finalstd'], 'g', lw=4, label=r'1982-2011 mean +$5\sigma_{final}$')

ax.set_xlim(0,365)
plt.show()

## old graph

In [ ]:
fig,(ax, ax1) = plt.subplots(2,1,  sharex=True, figsize=(16,14), height_ratios=[.75,.25], num=1, clear=True)

ax.plot(cdf.index,cdf['mean'], 'b', lw=4, label='1982-2011 mean')
ax.plot(cdf.index, cdf['mean+2std'], 'k', lw=4, label=r'1982-2011 mean $\pm 2\sigma$')
ax.plot(cdf.index, cdf['mean+5std'], 'r', lw=4, label=r'1982-2011 mean +$5\sigma$')
ax.plot(cdf.index, cdf['mean-2std'], 'k', lw=4)


_skip = 10
ms=5
hl_years = [('2009', 'ko'), ('2010','k+'), ('2011','k^'), ('2012', 'ro'), ('2013','r+'), ('2014','r^'), ('2023','rs'), ('2024','rD')]
for _year,fmt  in hl_years:
  day_num = cdf.index.to_list()[0::_skip]
  data = cdf[_year].iloc[0::_skip]
  ax.plot(day_num, data, fmt, ms=ms, label=_year)

yearL = range(1981, int(last_year)+1)
for _year in yearL:
  label=None
  if _year<=2011:
    fmt = 'k'
    if _year==2011:
      label = '1981-2011 data'
  else:
    fmt='r'
    if _year==int(last_year):
      label=f'2012-{last_year} data'
  ax.plot(cdf.index, cdf.loc[:,f'{_year}'],  fmt, lw=.5, label=label)


ax.set_ylabel(r'$T\ \  (^{\circ} C)$', fontsize='xx-large')
#ax.set_xlabel(r'day \#', fontsize='xx-large')
ax.set_title(f'1981-{last_year} SST daily average', fontsize='xx-large')
ax.text(295, 19.6,  f'\\it{{data current to {last_date}}}', fontsize='xx-large')
ax.set_xlim(0,365)
ax.legend(ncols=5)

ax1.plot(cdf.index,cdf['std'], 'b.', ms=4)
ax1.set_ylabel(r'$\sigma\ \  (^{\circ} C)$', fontsize='xx-large')
ax1.set_xlabel(r'day \#', fontsize='xx-large')
ax1.set_title(r'daily $\sigma$', fontsize='xx-large')
plt.tight_layout()
plt.savefig('SST-black-red.png')
plt.show()

## color schemes

In [ ]:
# Create figure and adjust figure height to number of colormaps

ncols=len(cmap_types)
nrows = max([len(arr) for arr in cmaps.values()])
cbarfig, axsL = plt.subplots(nrows=nrows, ncols=ncols, sharey=True, sharex=True, figsize=(16, 16), num=3, clear=True)
cbarfig.subplots_adjust(hspace=0.35, wspace=0.05)

for axrow in axsL:
  for ax in axrow:
    # Turn off *all* ticks & spines, not just the ones with colormaps.
    ax.set_axis_off()

for n, key in zip(range(ncols), cmaps.keys()):
  axs = [[j,axsL[j][n]]  for j in range(nrows)]
  for (j, ax), cmap_name in zip(axs, cmaps[key]):
    if j==0:
      ax.text(.5, 1.05, cmap_types[n], ha='center', fontsize=12, transform=ax.transAxes)
    ax.imshow(gradient, aspect='auto', cmap=cmap_name)
    ax.text(.5, -0.3, cmap_name, ha='center', fontsize=12, transform=ax.transAxes)
    # turn on just the box outline
    ax.set_axis_on()
    ax.set_xticks([])
    ax.set_yticks([])

cbarfig